In [3]:
import pandas as pd
import numpy as np
import re
import libsbml
from libsbml import *
from decimal import Decimal

use_pysb_sbml_translator=False # use optional sbml translator integrated with pySB based on the Atomizer (https://ruleworld.github.io/atomizer/blog/basic/bng.html)
if use_pysb_sbml_translator:
    import pysb
    from pysb.importers.sbml import sbml_translator

# select an example SBML model for conversion to bngl:
# 'Nair_2016'   - the main usecase model, https://github.com/jpgsantos/Model_Nair_2016
# 'Viswan_2018' - supplementary usecase model corresponding to Viswan_2018_optimized.xml from https://github.com/jpgsantos/Model_Viswan_2018
# 'Fujita_2010' - supplementary usecase model, https://github.com/jpgsantos/Model_Fujita_2010
do_model = 'Nair_2016' #{ 'Nair_2016' , 'Fujita_2010' , 'Viswan_2018' }

if do_model == 'Nair_2016':
    dxml = '' #'./examples/Nair_2016/'
    fxml = 'Nair_2016_optimized.xml'
    #fxml = 'Nair_2016_STEPS.xml'
    #adapt_steps={'list_of_steps':['transform_names','transform_units','clamp_species']}
    adapt_steps={'list_of_steps':['transform_names','transform_units']} # optional additional model adaptation steps needed to make it compatible with bngl

elif do_model == 'Fujita_2010':
    dxml = './examples/Fujita_2010/'
    fxml = 'Fujita_2010.xml'
    adapt_steps={'list_of_steps':['transform_names','transform_units','clamp_species']}
elif do_model == 'Viswan_2018':   
    dxml = './examples/Viswan_2018/'
    fxml = 'Viswan_2018_for_STEPS_optimised.xml'
    adapt_steps={'list_of_steps':['transform_names','transform_units','clamp_species']}

import importlib
import sbml_to_bngl
importlib.reload(sbml_to_bngl)

fbngl2 = dxml+fxml.replace('.xml', '.bngl')  # output file name

converter='plain' # select converter from sbml to bngl : 
#
#                  if converter='plain' - a libsbml based convertor 
#                    for sbml level 2, version 4 will be used. This converter produces a bngl approximation to
#                    reaction network format of a model. It is assumed that sbml models were obtained by importing
#                    a MATLAB simbiology model to sbml, or from SBTAB model by sbtab_to_vfgen.R converter.
#
#                  if converter='pysb' - a convertor based on the atomizer implemented by pysb.importers.sbml.sbml_translator()
#                    function will be used. The atomizer (https://ruleworld.github.io/atomizer/blog/basic/bng.html)
#                    will modify the set of model molecules and reactions to convert the model from reaction network to
#                    rule based format


# adapt_steps={'list_of_steps':['transform_names','transform_units','clamp_species']} - optional additional
#                                             model adaptation steps needed to make it compatible with bngl
#
# Models expressed by SBML and SBTAB often are not fully competible with BNGL.
# Additional model adaptation steps are required in this case to obtain a working BNGL model. 
# These steps will be partially automatized by sbml_to_bngl.transform function
# if adapt_steps dictionary 'list_of_steps' is nonempty.
#
# Implemented adaptation steps include:
#
# 1) STEPS and NFsim require different units for species quantities an kinetic rates, adapted bngl model provides modifed
#    expressions for all species concentrations and kinetic rates and provides
#    an easy way for changing units by specification of several auxilary bngl model parameters: Na and V_comparment_name,
#    These parameters should be selected to: Na=6.022e23 and V_comparment_name to volume of corresponding compartment in 
#    liters for NFsim and to: Na=1 and V_comparment_name=1 for STEPS. 
#
# 2) species with fixed concentrations are not supported by NFsim and STEPS solvers. bngl model adaptation will
#    modify model reactions such that a fixed species concentration became a model parameter. This parameter can be used
#    to fix species concentration or to apply external stimulation
#
# 3) if sbml to bngl convertor implemented in atomizer 
#    is selected (by selection converter='pysb') than additional transformation steps implemented in adapt_steps
#    includes renaming of duplicated molecule sites and reparing incorrect molecule names and kinetic rates transformations
#
# 4) in case when MATLAB simbiology is used for sbml model creation, adapt_steps will repare incorrect molecule and parameter names
#
# 5) compartmental model of BNGL assumes tree structure of the set of model comartments. This assumption is often
#    incompatible with mesh geometries supported by STEPS. adapt_steps produces bngl models with flexible compartmental structure
#
# 6) There are a number of incomatibilities between SBTAB and BNGL which still require manual correction, such as concentrations
#    fixed to an expression, functional expressions for reaction rates in case of STEPS solver, some nonstandard types
#    of reaction kinetic functions etc. The adapt_steps detects the cases of known incompatibilities and produces 
#    corresponding warning messages 


dsbml, mdl = sbml_to_bngl.transform(dxml+fxml, fbngl2, adapt_steps=adapt_steps, converter='plain')   

read document ...
read units ...
read compartments ...
read functions...
read species...

<Species DA_expression "DA_expression">
read parameters...
read reactions...
write plain bngl...
done
to select between steps and nfsim otput mode set adapt_steps['output_mode']='steps', otherwise steps output bngl file will be produced
save modified bngl file...
done


### Read and display the BNGL model

In [4]:
fbngl2 = re.split('\.bngl',fbngl2)[0]+'_alternative.bngl'
with open(fbngl2, 'r') as file:
    #data = file.read().replace('\n', '')
    sbngl = file.read()
    
print(sbngl)    

begin model
begin parameters
Na 1.0 # set Na to 6.022e23 for NFsim, or to 1.0 for STEPS
v_Spine 1.0 # set to 1.0 for STEPS or to volume in liters for NFsim
Na_x_v_Spine Na*v_Spine
k2k_m12_1xSpine_1xSpine_ 1/1e-12*1/(Na_x_v_Spine) # transformation of kinetic rates units, final units should be descendants of mole/liter, second for STEPS and number of molecules, second for NFsim, CHECK IT
k2k_m21_1xSpine_2xSpine_ 1/1e-21*1/(Na_x_v_Spine)*1/(Na_x_v_Spine) # transformation of kinetic rates units, final units should be descendants of mole/liter, second for STEPS and number of molecules, second for NFsim, CHECK IT
k2k_m3_1xSpine_ 1/1e-3 # transformation of kinetic rates units, final units should be descendants of mole/liter, second for STEPS and number of molecules, second for NFsim, CHECK IT
c2c_0_Spine 1e0*(Na_x_v_Spine) # transformation of concentration units,final units should be descendants of mole/liter for STEPS and number of molecules for NFsim, CHECK IT
c2c_m9_Spine 1e-9*(Na_x_v_Spin